#### Load the Data

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/features_btc.csv', parse_dates=['Date'])
df = df.sort_values('Date')
df = df.reset_index(drop=True)

# Just to visualize
df[['Date', 'Close']].tail()


,Date,Close
1789,2025-04-03,83102.828125
1790,2025-04-04,83843.804688
1791,2025-04-05,83504.796875
1792,2025-04-06,78214.484375
1793,2025-04-07,79235.335938


### Data Preparation for LSTM Model

In [3]:
from sklearn.preprocessing import MinMaxScaler

# Choose features (drop Date + Close for now)
features = df.drop(columns=['Date', 'Close'])
target = df['Close']

# Normalize features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Turn into sequences for LSTM
def create_sequences(X, y, window=10):
    X_seq, y_seq = [], []
    for i in range(len(X) - window):
        X_seq.append(X[i:i+window])
        y_seq.append(y[i+window])
    return np.array(X_seq), np.array(y_seq)

window_size = 10
X, y = create_sequences(scaled_features, target.values, window=window_size)

X.shape, y.shape  # Should be (num_samples, 10, num_features)


((1784, 10, 13), (1784,))

### Build and Train the LSTM Model

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(X.shape[1], X.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train/Test Split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))


ModuleNotFoundError: No module named 'tensorflow'